# Document Similarity

## Analyse data

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from matplotlib import style
style.use('ggplot')

plt.rc('figure', figsize=(40, 20))

from matplotlib import rcParams
rcParams['font.size'] = 30
rcParams['axes.unicode_minus'] = False


### Define value

In [ ]:
fname_result = 'data/result_score.pkl'

### Load contents file (pickle)

In [ ]:
df_c = pd.read_pickle(fname_result)

In [ ]:
df_c.columns

In [ ]:
df_c.loc[df_c['class_type'] == 'B','class_type'] = 'BEFORE'
df_c.loc[df_c['class_type'] == 'A','class_type'] = 'AFTER'

### Normalization

In [ ]:
col_list1 = list(df_c.columns[:5])
col_list2 = list(df_c.columns[5:])

In [ ]:
MAX = df_c[col_list2].max().max()
MAX

In [ ]:
MIN = df_c[col_list2].min().min()
MIN

In [ ]:
def func(x):
    return (x - MIN) * 100 / (MAX - MIN)

In [ ]:
df_n = pd.concat([df_c[col_list1], df_c[col_list2].applymap(func)], axis=1)

## 1. 기업(comp)별 집계

### 기업별 분류 컬럼 추가

In [ ]:
df_n['comp'] = df_n['fname'].str.split('_').apply(lambda x: x[0])

### 년도 분류 컬럼 추가

In [ ]:
df_n['year'] = df_n['fname'].str.split(r'_|\(|\-|\.').apply(lambda x: x[1])

In [ ]:
df_n

### 기업별 Goal 점수(유사도 평균) 계산

In [ ]:
gp_comp = df_n.groupby(['class_type','comp']).mean(numeric_only=True)
gp_comp.drop('doc_id', axis=1, inplace=True)

In [ ]:
gp_comp

### 기업별 Goal 점수: Before & After 비교 - comp 기준

In [ ]:
gp_comp_T = gp_comp.swaplevel('class_type','comp').T.sort_index(level=0,axis=1)

In [ ]:
gp_comp_T

In [ ]:
# 기업 리스트
comp_list = ['Apple', 'BASF', 'CocaCola', 'CooperativeBank', 'Danone', 'GE',
             'IKEA',  'Interface', 'MS', 'NIKE', 'Natura', 'Nestle', 'NovoNordisk',
             'Patagonia', 'Puma', 'Siemens', 'Tesla', 'Toyota', 'Unilever', 'Walmart']

### Chart - 기업별 Goal 점수: Before & After 비교 - 기업 기준

In [ ]:
# 기업명 설정
comp_name = comp_list[0]

gp_comp_T[comp_name].plot(kind='bar', rot=10, alpha=0.8)
plt.title(f'< {comp_name} >')
plt.show()

In [ ]:
# 기업명 설정
comp_name = comp_list[1]

gp_comp_T[comp_name].plot(kind='bar', rot=10, alpha=0.8)
plt.title(f'< {comp_name} >')
plt.show()

### 기업별 Goal 점수: Before & After 비교 - class_type 기준

In [ ]:
gp_comp.T

### Chart - 기업별 Goal 점수: Before & After 비교 - class_type 기준

In [ ]:
# class_type 설정
c_type = 'BEFORE'

gp_comp.T[c_type].plot(kind='bar', rot=10, alpha=0.8)
plt.title(f'< {c_type} >')
plt.show()

In [ ]:
# class_type 설정
c_type = 'AFTER'

gp_comp.T[c_type].plot(kind='bar', rot=10, alpha=0.8)
plt.title(f'< {c_type} >')
plt.show()

---

## 2. 문서(doc_id)별 집계

### 문서별 Goal별 유사도 분포 - Before, After 비교

### 문서별 Goal 점수(유사도 평균) 계산

In [ ]:
gp_doc = df_n.groupby(['class_type','doc_id']).mean(numeric_only=True)

In [ ]:
gp_doc

### Goal별 유사도 평균

In [ ]:
# Before
sr_B = gp_doc.loc['BEFORE'].apply(lambda x: x.mean())

# After
sr_A = gp_doc.loc['AFTER'].apply(lambda x: x.mean())

df1 = pd.concat([sr_B, sr_A], axis=1)
df1.columns = ['Before', 'After']

#### Before 순위

In [ ]:
df1.sort_values('Before', ascending=False)

#### After 순위

In [ ]:
df1.sort_values('After', ascending=False)

### Chart - Goal별 유사도 평균

In [ ]:
df1.plot.bar(rot=0, alpha=0.8)
plt.ylim(8,14)

#plt.title('Goal별 유사도 평균')
#plt.xlabel('Group')
#plt.ylabel('Score')

#plt.savefig('score_brazil.jpeg')

plt.show()

### Goal별 유사도 1위 비율

In [ ]:
# Before
sr_rB = gp_doc.loc['BEFORE'].apply(lambda x: x.idxmax(), axis=1).value_counts() * 100 / len(gp_doc.loc['BEFORE'])

# After
sr_rA = gp_doc.loc['AFTER'].apply(lambda x: x.idxmax(), axis=1).value_counts() * 100 / len(gp_doc.loc['AFTER'])

df2 = pd.concat([sr_rB, sr_rA], axis=1)
df2.columns = ['Before', 'After']

#### Before 순위

In [ ]:
df2.sort_values('Before', ascending=False)

#### After 순위

In [ ]:
df2.sort_values('After', ascending=False)

### Chart - Goal별 유사도 1위 비율

In [ ]:
df2.plot.bar(rot=0, alpha=0.8)
plt.show()

---

In [ ]:
# End of file